In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('synthetic_instagram_plg_data.csv')
data.head()

,user_id,signup_date,age,gender,location,daily_active_users,content_views,likes_given,comments_made,shares,...,topic_preferences,sponsored_content_engagement,churned,returning_user,viral_invites_sent,time_to_first_post,time_to_first_story,test_group,conversion_rate,engagement_increase
0,1,2023-11-24,42,Male,Australia,22,434,288,106,195,...,['Tech'],6,True,True,5,20.4,0.8,Variant B,0.03,-0.07
1,2,2022-07-17,55,Female,UK,9,704,158,118,137,...,"['Music', 'Movies', 'Fashion', 'Politics']",22,True,False,9,15.8,11.3,Variant A,0.20,0.38
2,3,2023-09-05,42,Male,Australia,30,2469,412,55,221,...,['Food'],28,False,True,0,26.9,23.3,Variant A,0.19,0.38
3,4,2024-01-18,23,Female,Japan,28,997,105,106,432,...,"['Art', 'Health', 'Gaming', 'Business', 'Polit...",30,True,True,7,19.1,28.3,Variant B,0.18,0.48
4,5,2023-04-25,54,Male,Canada,16,2209,412,133,117,...,"['Politics', 'Science', 'Travel', 'Fitness', '...",10,True,False,1,14.6,24.6,Control,0.19,0.28


### Data Transformation

In [3]:
data.dtypes

user_id                                      int64
signup_date                                 object
age                                          int64
gender                                      object
location                                    object
daily_active_users                           int64
content_views                                int64
likes_given                                  int64
comments_made                                int64
shares                                       int64
direct_messages_sent                         int64
posts_created                                int64
stories_created                              int64
reels_created                                int64
followers_count                              int64
following_count                              int64
uses_explore                                  bool
uses_shopping                                 bool
uses_igtv                                     bool
uses_reels                     

In [4]:
typeChange_columns = []
for i in data.columns:
    if data[i].dtype == object:
        typeChange_columns.append(i)

In [5]:
typeChange_columns

['signup_date',
 'gender',
 'location',
 'apps_and_websites_type_off_of_instagram',
 'topic_preferences',
 'test_group']

In [6]:
data['signup_date'] = pd.to_datetime(data['signup_date'])

In [7]:
data['gender'] = pd.Categorical(data['gender'])

In [8]:
data['location'] = pd.Categorical(data['location'])

In [9]:
data['apps_and_websites_type_off_of_instagram'] = pd.Categorical(data['apps_and_websites_type_off_of_instagram'])

In [10]:
data['topic_preferences'] = data['topic_preferences'].tolist()

In [11]:
data['test_group'] = pd.Categorical(data['test_group'])

In [12]:
data.isna().sum()

user_id                                    0
signup_date                                0
age                                        0
gender                                     0
location                                   0
daily_active_users                         0
content_views                              0
likes_given                                0
comments_made                              0
shares                                     0
direct_messages_sent                       0
posts_created                              0
stories_created                            0
reels_created                              0
followers_count                            0
following_count                            0
uses_explore                               0
uses_shopping                              0
uses_igtv                                  0
uses_reels                                 0
uses_stories                               0
story_views                                0
reel_views

In [66]:
data.head()

,user_id,signup_date,age,gender,location,daily_active_users,content_views,likes_given,comments_made,shares,...,churned,returning_user,viral_invites_sent,time_to_first_post,time_to_first_story,test_group,conversion_rate,engagement_increase,time_to_value,total_engagements
0,1,2023-11-24,42,Male,Australia,22,434,288,106,195,...,True,True,5,20.4,0.8,Variant B,0.03,-0.07,0.8,1029
1,2,2022-07-17,55,Female,UK,9,704,158,118,137,...,True,False,9,15.8,11.3,Variant A,0.20,0.38,11.3,1139
2,3,2023-09-05,42,Male,Australia,30,2469,412,55,221,...,False,True,0,26.9,23.3,Variant A,0.19,0.38,23.3,3185
3,4,2024-01-18,23,Female,Japan,28,997,105,106,432,...,True,True,7,19.1,28.3,Variant B,0.18,0.48,28.3,1670
4,5,2023-04-25,54,Male,Canada,16,2209,412,133,117,...,True,False,1,14.6,24.6,Control,0.19,0.28,24.6,2881


In [14]:
# adding PLG metrics to data as new columns
#Time to Value
data['time_to_value'] = data['time_to_first_story']


In [40]:
feature_usage_rates = {}
for feature in ['uses_explore', 'uses_shopping', 'uses_igtv', 'uses_reels', 'uses_stories']:
    feature_usage_rates[feature] = (data[feature] == True).sum()/len(data[feature])

## Product Adoption Rate

In [41]:
feature_usage_rates

{'uses_explore': np.float64(0.50234),
 'uses_shopping': np.float64(0.49948),
 'uses_igtv': np.float64(0.49648),
 'uses_reels': np.float64(0.49656),
 'uses_stories': np.float64(0.50266)}

So I could tell that stories and explore features increased product adoption

## Retention Rate

In [42]:
retention_rates = {}
for feature in ['returning_user', 'churned']:
    retention_rates[feature] = (data[feature] == True).sum()/len(data[feature])

In [43]:
retention_rates

{'returning_user': np.float64(0.49944), 'churned': np.float64(0.49827)}

In [50]:
(data['daily_active_users'].sum()/30)/len(data['daily_active_users'])

np.float64(0.5168513333333333)

## so the retention rate is around 50-51% over a period of 30 days

### Engagement per user

In [68]:
def calculate_engagements_per_user(data):
  engagement_columns = ['content_views', 'likes_given', 'comments_made', 'shares', 'sponsored_content_engagement']

  data['total_engagements'] = data[engagement_columns].sum(axis=1)

  # Group by user and calculate aggregate and average engagements
  user_engagements = data.groupby('user_id')[['total_engagements']].agg(['sum', 'mean'])
  user_engagements.columns = ['total_engagements', 'average_engagements']

  return user_engagements

result = calculate_engagements_per_user(data)
result

,total_engagements,average_engagements
user_id,,
1,1029,1029.0
2,1139,1139.0
3,3185,3185.0
4,1670,1670.0
5,2881,2881.0
...,...,...
99996,1262,1262.0
99997,1428,1428.0
99998,2414,2414.0


## CLTV

In [69]:
def calculate_total_revenue(data):
  data['ad_revenue'] = data['ads_clicked'] * 0.40
  # Calculate total revenue
  data['CLTV'] = data['in_app_purchases'] + data['ad_revenue']

  # Group by user and calculate total revenue
  user_revenue = data.groupby('user_id')['CLTV'].sum()

  return user_revenue

result = calculate_total_revenue(data)
result

user_id
1          21.11
2         425.21
3         391.97
4         301.97
5         363.19
           ...  
99996     419.84
99997     406.26
99998     231.01
99999      60.90
100000     83.58
Name: CLTV, Length: 100000, dtype: float64

In [71]:
data.dtypes

user_id                                             int64
signup_date                                datetime64[ns]
age                                                 int64
gender                                           category
location                                         category
daily_active_users                                  int64
content_views                                       int64
likes_given                                         int64
comments_made                                       int64
shares                                              int64
direct_messages_sent                                int64
posts_created                                       int64
stories_created                                     int64
reels_created                                       int64
followers_count                                     int64
following_count                                     int64
uses_explore                                         bool
uses_shopping 

In [72]:
data.to_csv('insta_clean_data.csv', index=False)